In [551]:
# Подключаем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**1. Исходные данные**

In [552]:
df = pd.read_csv('/content/drive/MyDrive/t3_non_organic.csv', sep = ',')
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,Language,AppsFlyer ID,Advertising ID,IDFA,Platform,Device Type,OS Version,Attribution Lookback,Is Primary Attribution,User Agent
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,русский,1597410570684-3799928820535696190,2bca3e9d-1c06-4c13-8895-c311f9d8bfdd,NaN,android,xiaomi-Redmi Note 5,9,28d,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,русский,1597420480729-1759484924098917478,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,русский,1597420480729-1759484924098917478,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,русский,1597455446369-7748387263592684895,96acf210-19f7-4613-9259-8af44c5744ed,NaN,android,samsung-SM-T515,9,30d,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,English,1597501316454-1211890571372415119,48edfbd9-212a-477b-884e-46215c7ec482,NaN,android,nubia-NX659J,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...


**Первичный анализ данных. Подготовка данных.**

In [553]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59603 entries, 0 to 59602
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Attributed Touch Type   57612 non-null  object 
 1   Attributed Touch Time   57612 non-null  object 
 2   Install Time            59603 non-null  object 
 3   Event Time              59603 non-null  object 
 4   Event Name              59603 non-null  object 
 5   Event Value             59603 non-null  object 
 6   Event Revenue           7 non-null      float64
 7   Event Revenue Currency  59603 non-null  object 
 8   Event Revenue USD       7 non-null      float64
 9   Media Source            59603 non-null  object 
 10  Channel                 57612 non-null  object 
 11  Campaign                57612 non-null  object 
 12  Campaign ID             57612 non-null  float64
 13  Adset                   51975 non-null  object 
 14  Adset ID                51975 non-null

In [554]:
print(df['Event Revenue'].unique())

[   nan 219.99  10.99   8.29 279.99  12.99]


In [555]:
df.shape

(59603, 33)

**2. Основные задачи**

**2.1.** Собрать когортный отчет, где за когорту взять весь трафик (когорта от установки) в разбивке по стране.<br>
В качестве визуализации составить табличку в Google Sheets, в строках которой будут страны, а в столбцах набор метрик:<br>
a. Installs<br>
b. Payers - число платящих<br>
c. Purchases - число покупок<br>
d. Tutorial Finished - процент прохождения тутора<br>
e. level_10% - процент прохождения уровня<br>
f. C2p - процент платящих<br>
g. RPI - доход на одну установку<br>
h. Revenue - доход в абсолютном значении


In [556]:
print(df.columns.values)

['Attributed Touch Type' 'Attributed Touch Time' 'Install Time'
 'Event Time' 'Event Name' 'Event Value' 'Event Revenue'
 'Event Revenue Currency' 'Event Revenue USD' 'Media Source' 'Channel'
 'Campaign' 'Campaign ID' 'Adset' 'Adset ID' 'Ad' 'Ad ID' 'Ad Type'
 'Site ID' 'Region' 'Country Code' 'IP' 'WIFI' 'Language' 'AppsFlyer ID'
 'Advertising ID' 'IDFA' 'Platform' 'Device Type' 'OS Version'
 'Attribution Lookback' 'Is Primary Attribution' 'User Agent']


In [557]:
df['Event Name'].unique()

array(['af_content_view', 'LaunchApp', 'FirstLaunchApp',
       'af_tutorial_completion', 'af_purchase', 'achieve_level_10',
       'purchase_ta_starter', 'achieve_level_15',
       'purchase_ta_challenge_pass'], dtype=object)

In [558]:
df.sort_values('Country Code')
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,Language,AppsFlyer ID,Advertising ID,IDFA,Platform,Device Type,OS Version,Attribution Lookback,Is Primary Attribution,User Agent
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,русский,1597410570684-3799928820535696190,2bca3e9d-1c06-4c13-8895-c311f9d8bfdd,NaN,android,xiaomi-Redmi Note 5,9,28d,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,русский,1597420480729-1759484924098917478,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,русский,1597420480729-1759484924098917478,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,русский,1597455446369-7748387263592684895,96acf210-19f7-4613-9259-8af44c5744ed,NaN,android,samsung-SM-T515,9,30d,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,English,1597501316454-1211890571372415119,48edfbd9-212a-477b-884e-46215c7ec482,NaN,android,nubia-NX659J,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...


In [560]:
df[df['Event Revenue'].isna() == True]
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,Language,AppsFlyer ID,Advertising ID,IDFA,Platform,Device Type,OS Version,Attribution Lookback,Is Primary Attribution,User Agent
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,русский,1597410570684-3799928820535696190,2bca3e9d-1c06-4c13-8895-c311f9d8bfdd,NaN,android,xiaomi-Redmi Note 5,9,28d,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,русский,1597420480729-1759484924098917478,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,русский,1597420480729-1759484924098917478,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,русский,1597455446369-7748387263592684895,96acf210-19f7-4613-9259-8af44c5744ed,NaN,android,samsung-SM-T515,9,30d,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,English,1597501316454-1211890571372415119,48edfbd9-212a-477b-884e-46215c7ec482,NaN,android,nubia-NX659J,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...


In [561]:
df[df['IDFA'].isna() == True]
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,Language,AppsFlyer ID,Advertising ID,IDFA,Platform,Device Type,OS Version,Attribution Lookback,Is Primary Attribution,User Agent
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,русский,1597410570684-3799928820535696190,2bca3e9d-1c06-4c13-8895-c311f9d8bfdd,NaN,android,xiaomi-Redmi Note 5,9,28d,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,русский,1597420480729-1759484924098917478,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,русский,1597420480729-1759484924098917478,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,русский,1597455446369-7748387263592684895,96acf210-19f7-4613-9259-8af44c5744ed,NaN,android,samsung-SM-T515,9,30d,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,English,1597501316454-1211890571372415119,48edfbd9-212a-477b-884e-46215c7ec482,NaN,android,nubia-NX659J,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...


In [562]:
df['Installs'] = df.groupby('Country Code')['Install Time'].transform('count')
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,AppsFlyer ID,Advertising ID,IDFA,Platform,Device Type,OS Version,Attribution Lookback,Is Primary Attribution,User Agent,Installs
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,1597410570684-3799928820535696190,2bca3e9d-1c06-4c13-8895-c311f9d8bfdd,NaN,android,xiaomi-Redmi Note 5,9,28d,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...,29556
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,1597420480729-1759484924098917478,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,1597420480729-1759484924098917478,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,1597455446369-7748387263592684895,96acf210-19f7-4613-9259-8af44c5744ed,NaN,android,samsung-SM-T515,9,30d,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...,29556
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,1597501316454-1211890571372415119,48edfbd9-212a-477b-884e-46215c7ec482,NaN,android,nubia-NX659J,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...,4655


In [563]:
df['Payers'] = df.groupby('Country Code')['Event Revenue'].transform('count')
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,Advertising ID,IDFA,Platform,Device Type,OS Version,Attribution Lookback,Is Primary Attribution,User Agent,Installs,Payers
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,2bca3e9d-1c06-4c13-8895-c311f9d8bfdd,NaN,android,xiaomi-Redmi Note 5,9,28d,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...,29556,3
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,a9423486-6065-42ac-9b75-54c712da3435,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,96acf210-19f7-4613-9259-8af44c5744ed,NaN,android,samsung-SM-T515,9,30d,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...,29556,3
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,48edfbd9-212a-477b-884e-46215c7ec482,NaN,android,nubia-NX659J,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...,4655,3


In [564]:
df['Purchases'] = df.groupby('Country Code')['Event Revenue'].transform('count')
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,IDFA,Platform,Device Type,OS Version,Attribution Lookback,Is Primary Attribution,User Agent,Installs,Payers,Purchases
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,NaN,android,xiaomi-Redmi Note 5,9,28d,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...,29556,3,3
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,NaN,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,NaN,android,samsung-SM-T515,9,30d,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...,29556,3,3
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,NaN,android,nubia-NX659J,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...,4655,3,3


In [565]:
df['Revenue'] =round((df.groupby('Country Code')['Event Revenue USD'].transform('sum')), 2)
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,Platform,Device Type,OS Version,Attribution Lookback,Is Primary Attribution,User Agent,Installs,Payers,Purchases,Revenue
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,android,xiaomi-Redmi Note 5,9,28d,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...,29556,3,3,26.34
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1,8.29
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,android,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1,8.29
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,android,samsung-SM-T515,9,30d,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...,29556,3,3,26.34
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,android,nubia-NX659J,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...,4655,3,3,26.40


In [566]:
df['RPI'] = round((df['Revenue']/df['Installs']),4)
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,Device Type,OS Version,Attribution Lookback,Is Primary Attribution,User Agent,Installs,Payers,Purchases,Revenue,RPI
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,xiaomi-Redmi Note 5,9,28d,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...,29556,3,3,26.34,0.0009
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1,8.29,0.0011
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,Redmi-Redmi Note 9 Pro,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1,8.29,0.0011
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,samsung-SM-T515,9,30d,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...,29556,3,3,26.34,0.0009
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,nubia-NX659J,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...,4655,3,3,26.40,0.0057


In [567]:
df['C2p'] = round(((df['Payers']/ df['Installs'])*100),4)
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,OS Version,Attribution Lookback,Is Primary Attribution,User Agent,Installs,Payers,Purchases,Revenue,RPI,C2p
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,9,28d,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...,29556,3,3,26.34,0.0009,0.0102
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1,8.29,0.0011,0.0128
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1,8.29,0.0011,0.0128
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,9,30d,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...,29556,3,3,26.34,0.0009,0.0102
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,10,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...,4655,3,3,26.40,0.0057,0.0644


In [568]:
df['level_10%'] = round((((len(df[df['Event Name']=='achieve_level_10']))/df['Installs'])),0)
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,Attribution Lookback,Is Primary Attribution,User Agent,Installs,Payers,Purchases,Revenue,RPI,C2p,level_10%
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,28d,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...,29556,3,3,26.34,0.0009,0.0102,0.0
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1,8.29,0.0011,0.0128,0.0
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1,8.29,0.0011,0.0128,0.0
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,30d,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...,29556,3,3,26.34,0.0009,0.0102,0.0
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,28d,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...,4655,3,3,26.40,0.0057,0.0644,0.0


In [569]:
df['Tutorial Finished'] = round((((len(df[df['Event Name']=='af_tutorial_completion'])*0.1)/df['Installs'])),0)
df.head()

,Attributed Touch Type,Attributed Touch Time,Install Time,Event Time,Event Name,Event Value,Event Revenue,Event Revenue Currency,Event Revenue USD,Media Source,...,Is Primary Attribution,User Agent,Installs,Payers,Purchases,Revenue,RPI,C2p,level_10%,Tutorial Finished
0,click,2020-08-14 12:14:22,2020-08-14 13:09:32,2020-08-15 23:59:53,af_content_view,"{""af_content"":"""",""af_content_type"":""user-chat""}",NaN,USD,NaN,Facebook Ads,...,True,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...,29556,3,3,26.34,0.0009,0.0102,0.0,0.0
1,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:47,af_content_view,"{""af_content"":""point:40"",""af_content_type"":""qu...",NaN,USD,NaN,Facebook Ads,...,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1,8.29,0.0011,0.0128,0.0,0.0
2,click,2020-08-12 19:41:19,2020-08-14 15:54:44,2020-08-15 23:59:43,af_content_view,"{""af_content"":""step:2"",""af_content_type"":""mini...",NaN,USD,NaN,Facebook Ads,...,True,Dalvik/2.1.0 (Linux; U; Android 10; Redmi Note...,7787,1,1,8.29,0.0011,0.0128,0.0,0.0
3,click,2020-08-15 01:34:25,2020-08-15 01:37:29,2020-08-15 23:59:42,af_content_view,"{""af_content"":""mode:1vs1Rank"",""af_content_type...",NaN,USD,NaN,googleadwords_int,...,True,Dalvik/2.1.0 (Linux; U; Android 9; SM-T515 Bui...,29556,3,3,26.34,0.0009,0.0102,0.0,0.0
4,click,2020-08-15 14:03:15,2020-08-15 14:21:59,2020-08-15 23:59:42,af_content_view,"{""af_content"":""name:Imp level:2"",""af_content_t...",NaN,USD,NaN,Facebook Ads,...,True,Dalvik/2.1.0 (Linux; U; Android 10; NX659J Bui...,4655,3,3,26.40,0.0057,0.0644,0.0,0.0


In [ ]:
df = df.drop(['Ad ID', 'Adset ID', 'Campaign ID','Event Revenue', 'Event Revenue USD', 'Is Primary Attribution', 'WIFI'], axis=1)
df.head()

In [572]:
df.pivot_table(index='Country Code')

<ipython-input-572-6b1c071ec83b>:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  df.pivot_table(index='Country Code')


,C2p,Installs,Payers,Purchases,RPI,Revenue,Tutorial Finished,level_10%
Country Code,,,,,,,,
AF,0.0000,9,0,0,0.0000,0.00,4.0,3.0
AM,0.0000,415,0,0,0.0000,0.00,0.0,0.0
AZ,0.0000,1224,0,0,0.0000,0.00,0.0,0.0
BG,0.0000,1,0,0,0.0000,0.00,40.0,23.0
BY,0.0128,7787,1,1,0.0011,8.29,0.0,0.0
CA,0.0644,4655,3,3,0.0057,26.40,0.0,0.0
CZ,0.0000,22,0,0,0.0000,0.00,2.0,1.0
DE,0.0000,35,0,0,0.0000,0.00,1.0,1.0
DK,0.0000,1,0,0,0.0000,0.00,40.0,23.0


**2.2.** Проанализировать отчёт и сделать выводы по наблюдаемым метрикам.

Таким образом, можно сделать следующие выводы: наибольшее число установок игры в регионе с кодом UA (29556 установок), наименьшее число установок в регионах с кодом BG, DK, ES, TM, US (меньше 5 установок). Также можно сделать вывод, что число платящих пользователей крайне мало. И число пользователей прошедших 10 уровень также крайне мало, по сравнению с числом пользователей прошедщих обучение.
